In [1]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['dma_project'] 
patient_collection = db['Patient']

# Select all patients
patients = patient_collection.find({})

# Display the documents
for patient in patients:
    print(patient)

# Close the MongoDB connection
client.close()


{'_id': ObjectId('656bd6ac74083851072de8c0'), 'Name': 'John Doe', 'DateOfBirth': datetime.datetime(1990, 1, 1, 0, 0), 'Contact': '123-456-7890'}
{'_id': ObjectId('656bd8c974083851072de8c1'), 'PatientID': 1, 'Name': 'John Doe', 'DateOfBirth': datetime.datetime(1990, 5, 15, 0, 0), 'Contact': '123-456-7890', 'Insurance': 'ABC Insurance', 'MedicalHistory': 'No significant medical history'}
{'_id': ObjectId('656bd8d874083851072de8c2'), 'PatientID': 2, 'Name': 'Jane Smith', 'DateOfBirth': datetime.datetime(1985, 8, 22, 0, 0), 'Contact': '987-654-3210', 'Insurance': 'XYZ Insurance', 'MedicalHistory': 'Allergies to peanuts'}
{'_id': ObjectId('656bd8e474083851072de8c3'), 'PatientID': 3, 'Name': 'Alice Johnson', 'DateOfBirth': datetime.datetime(1978, 3, 10, 0, 0), 'Contact': '555-123-4567', 'Insurance': 'LMN Insurance', 'MedicalHistory': 'Hypertension'}
{'_id': ObjectId('656bd8ed74083851072de8c4'), 'PatientID': 4, 'Name': 'Bob Williams', 'DateOfBirth': datetime.datetime(1995, 11, 28, 0, 0), 'Con

In [9]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['dma_project']

# Retrieve information about patients and their associated doctors using INNER JOIN
print("Retrieve information about patients and their associated doctors using INNER JOIN")
result = db.Patient.aggregate([
    {
        '$lookup': {
            'from': 'Appointment',
            'localField': 'PatientID',
            'foreignField': 'PatientID',
            'as': 'appointments'
        }
    },
    {
        '$unwind': '$appointments'
    },
    {
        '$lookup': {
            'from': 'Doctor',
            'localField': 'appointments.DoctorID',
            'foreignField': 'DoctorID',
            'as': 'doctor'
        }
    },
    {
        '$unwind': '$doctor'
    },
    {
        '$project': {
            'PatientID': 1,
            'PatientName': '$Name',
            'DoctorID': '$doctor.DoctorID',
            'DoctorName': '$doctor.Name',
            'Specialization': '$doctor.Specialization'
        }
    }
])

for document in result:
    print(document)
print("\n")

# Patients who have appointments with doctors specializing in Cardiology
print("Patients who have appointments with doctors specializing in Cardiology")
result = db.Patient.aggregate([
    {
        '$lookup': {
            'from': 'Appointment',
            'localField': 'PatientID',
            'foreignField': 'PatientID',
            'as': 'appointments'
        }
    },
    {
        '$unwind': '$appointments'
    },
    {
        '$lookup': {
            'from': 'Doctor',
            'localField': 'appointments.DoctorID',
            'foreignField': 'DoctorID',
            'as': 'doctor'
        }
    },
    {
        '$unwind': '$doctor'
    },
    {
        '$match': {
            'doctor.Specialization': 'Cardiology'
        }
    },
    {
        '$project': {
            'PatientID': 1,
            'PatientName': '$Name',
            'AppointmentID': '$appointments.AppointmentID',
            'Date': '$appointments.Date',
            'Specialization': '$doctor.Specialization'
        }
    }
])

for document in result:
    print(document)
print("\n")

# Distribution of appointments across doctors
print("Distribution of appointments across doctors")
result = db.Doctor.aggregate([
    {
        '$lookup': {
            'from': 'Appointment',
            'localField': 'DoctorID',
            'foreignField': 'DoctorID',
            'as': 'appointments'
        }
    },
    {
        '$project': {
            'Name': 1,
            'AppointmentCount': {'$size': '$appointments'}
        }
    }
])

for document in result:
    print(document)
print("\n")

# Patients across different age groups
print("Patients across different age groups")
result = db.Patient.aggregate([
    {
        '$project': {
            'AgeGroup': {
                '$switch': {
                    'branches': [
                        {'case': {'$and': [{'$gte': ['$DateOfBirth', '2003-01-01']}, {'$lt': ['$DateOfBirth', '2013-01-01']}]}, 'then': '0-10'},
                        {'case': {'$and': [{'$gte': ['$DateOfBirth', '1993-01-01']}, {'$lt': ['$DateOfBirth', '2003-01-01']}]}, 'then': '11-20'},
                        # Add more cases for other age groups
                    ],
                    'default': 'Unknown'
                }
            }
        }
    },
    {
        '$group': {
            '_id': '$AgeGroup',
            'PatientCount': {'$sum': 1}
        }
    }
])

for document in result:
    print(document)
# print("\n")

Retrieve information about patients and their associated doctors using INNER JOIN
{'_id': ObjectId('656bd8c974083851072de8c1'), 'PatientID': 1, 'PatientName': 'John Doe', 'DoctorID': 1, 'DoctorName': 'Dr. Smith', 'Specialization': 'Cardiologist'}
{'_id': ObjectId('656bd8d874083851072de8c2'), 'PatientID': 2, 'PatientName': 'Jane Smith', 'DoctorID': 3, 'DoctorName': 'Dr. Brown', 'Specialization': 'Pediatrician'}
{'_id': ObjectId('656bd8d874083851072de8c2'), 'PatientID': 2, 'PatientName': 'Jane Smith', 'DoctorID': 2, 'DoctorName': 'Dr. Johnson', 'Specialization': 'Orthopedic Surgeon'}
{'_id': ObjectId('656bd8d874083851072de8c2'), 'PatientID': 2, 'PatientName': 'Jane Smith', 'DoctorID': 6, 'DoctorName': 'Dr. Wilson', 'Specialization': 'Neurologist'}
{'_id': ObjectId('656bd8e474083851072de8c3'), 'PatientID': 3, 'PatientName': 'Alice Johnson', 'DoctorID': 3, 'DoctorName': 'Dr. Brown', 'Specialization': 'Pediatrician'}
{'_id': ObjectId('656bd8ed74083851072de8c4'), 'PatientID': 4, 'PatientName